# Strategy MACD + RSI SMA in the Spot Market

In [ ]:
class Strategy(StrategyBase):
    def __init__(self):
        # strategy property
        self.subscribed_books = {}
        self.period = 60 * 60 
        self.options = {}

        self.last_type = 'sell'
        self.fast_period = 10
        self.slow_period = 20
        self.signal_period = 9
        self.proportion = 0.9
        self.rsi_period = 10

    def on_order_state_change(self,  order):
        pass

    # called every self.period
    def trade(self, candles):
        exchange, pair, base, quote = CA.get_exchange_pair()
        
        close_price_history = [candle['close'] for candle in candles[exchange][pair]]
        high_price_history = [candle['high'] for candle in candles[exchange][pair]]
        low_price_history = [candle['low'] for candle in candles[exchange][pair]]

        # convert to chronological order for talib
        close_price_history.reverse()
        high_price_history.reverse()
        low_price_history.reverse()

        # convert np.array
        close_price_history = np.array(close_price_history)
        high_price_history = np.array(high_price_history)
        low_price_history = np.array(low_price_history)

        if len(close_price_history) < 2:
            return []

        # get macd from talib
        macd, macdsignal, macdhist = talib.MACD(close_price_history, fastperiod=self.fast_period, slowperiod=self.slow_period, signalperiod=self.signal_period)

        curr_macd = macdhist[-1]
        prev_macd = macdhist[-2]

        macd_now = macd[-1]
        signal_now = macdsignal[-1]

        # get the simple moving average of the RSI value in the the specified period from talib
        all_rsi = talib.SMA(talib.RSI(close_price_history, self.rsi_period), self.rsi_period)
        
        curr_rsi = all_rsi[-1]

        signal = 0
        # MACD crosses the zero line from below - buy signal
        # RSI falls into the oversold region
        if (prev_macd < 0 and curr_macd > 0 and curr_rsi <= 30) :
            signal = 1
        
        # MACD crosses the zero line from above - sell signal
        # RSI rises into the overbought region
        elif (prev_macd > 0 and curr_macd < 0 and curr_rsi >= 70):
            signal = -1

        # get available balance
        base_balance = CA.get_balance(exchange, base)
        quote_balance = CA.get_balance(exchange, quote)
        available_base_amount = base_balance.available
        available_quote_amount = quote_balance.available

        # place buy order
        if self.last_type == 'sell' and signal == 1:
            amount = np.around((available_quote_amount /  close_price_history[-1]) * self.proportion, 5)
            self.last_type = 'buy'
            CA.log("Buy " + str(amount) + " " + base)
            
            if (amount > 0):
              CA.buy(exchange, pair, amount, CA.OrderType.MARKET)         

        # place sell order
        elif self.last_type == 'buy' and signal == -1:
            self.last_type = 'sell'

            if (available_base_amount > 0):
              CA.log("Sell " + str(available_base_amount) + " " + base)
              CA.sell(exchange, pair, available_base_amount, CA.OrderType.MARKET)
                
        return

# Strategy MACD + RSI SMA in the Futures Market

In [ ]:
class Strategy(StrategyBase):
    def __init__(self):
        # strategy property
        self.subscribed_books = {}
        self.period = 60 * 60 
        self.options = {}

        self.last_type = 'sell'
        self.fast_period = 10
        self.slow_period = 20
        self.signal_period = 9
        self.proportion = 0.9
        self.rsi_period = 10
 

    def on_order_state_change(self,  order):
        pass

    # called every self.period
    def trade(self, candles):
        exchange, pair, base, quote = CA.get_exchange_pair()
        
        close_price_history = [candle['close'] for candle in candles[exchange][pair]]
        high_price_history = [candle['high'] for candle in candles[exchange][pair]]
        low_price_history = [candle['low'] for candle in candles[exchange][pair]]

        # convert to chronological order for talib
        close_price_history.reverse()
        high_price_history.reverse()
        low_price_history.reverse()

        # convert np.array
        close_price_history = np.array(close_price_history)
        high_price_history = np.array(high_price_history)
        low_price_history = np.array(low_price_history)

        if len(close_price_history) < 2:
            return []

        # get macd from talib
        macd, macdsignal, macdhist = talib.MACD(close_price_history, fastperiod=self.fast_period, slowperiod=self.slow_period, signalperiod=self.signal_period)

        curr_macd = macdhist[-1]
        prev_macd = macdhist[-2]

        macd_now = macd[-1]
        signal_now = macdsignal[-1]

        # get the simple moving average of the RSI value in the the specified period from talib
        all_rsi = talib.SMA(talib.RSI(close_price_history, self.rsi_period), self.rsi_period)

        # curr_rsi = all_rsi[-1]
        curr_rsi = all_rsi[-1]

        signal = 0
        # MACD crosses the zero line from below - buy signal
        # RSI falls into the oversold region
        if (prev_macd < 0 and curr_macd > 0 and curr_rsi <= 30) :
            signal = 1
        
        # MACD crosses the zero line from above - sell signal
        # RSI rises into the overbought region
        elif (prev_macd > 0 and curr_macd < 0 and curr_rsi >= 70):
            signal = -1

        # get available balance
        base_balance = CA.get_balance(exchange, base)
        quote_balance = CA.get_balance(exchange, quote)
        available_base_amount = base_balance.available
        available_quote_amount = quote_balance.available

        # On buy signal
        if self.last_type == 'sell' and signal == 1:
            amount = np.around((available_quote_amount /  close_price_history[-1]) * self.proportion, 5)
            self.last_type = 'buy'

            # Close all short positions
            short_position = CA.get_position(exchange, pair, CA.PositionSide.SHORT)
            if short_position and short_position.available_size > 0:
                CA.log('Close all short positions for ' + base)
                CA.close_short(exchange, pair, short_position.available_size, CA.OrderType.MARKET)

            # Long the asset if available
            if (amount > 0):
                # Long
                CA.log('Long ' + str(amount) + ' ' + base)
                CA.open_long(exchange, pair, amount, CA.OrderType.MARKET)     

        # On sell signal
        elif self.last_type == 'buy' and signal == -1:
            amount = np.around((available_quote_amount /  close_price_history[-1]) * self.proportion, 5)
            self.last_type = 'sell'

            # Close all long positions
            long_position = CA.get_position(exchange, pair, CA.PositionSide.LONG)
            if long_position and long_position.available_size > 0:
                CA.log('Close all long positions for ' + base)
                CA.close_long(exchange, pair, long_position.available_size, CA.OrderType.MARKET)

            # Short the asset if available
            if (amount > 0):
                CA.log('Short ' + str(amount) + ' ' + base)
                CA.open_short(exchange, pair, amount, CA.OrderType.MARKET)
        return